### 1. combine `mon_features.pkl` & `unmon_features.pkl` into `features_df`

In [1]:
import pandas as pd
import numpy as np
import pickle

MON_FILE_PATH = '/content/mon_features.pkl'
UNMON_FILE_PATH = '/content/unmon_features.pkl'

LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER3 = [
    'total_transmission_time', 'std_inter_packet_time',
    'avg_outgoing_burst_size', 'avg_incoming_burst_size',
    'num_outgoing_packets', 'incoming_packet_ratio',
    'outgoing_packet_ratio', 'cumul_packets_10pct',
    'cumul_packets_30pct', 'outgoing_order_skew',
    'incoming_order_skew', 'cumul_max', 'bigram_OO',
    'num_incoming_first_30', 'outgoing_first_30',
    'avg_incoming_order_first_30', 'avg_outgoing_order_first_30'
]

mon_features_df = pd.read_pickle(MON_FILE_PATH)
unmon_features_df = pd.read_pickle(UNMON_FILE_PATH)

features_df = pd.concat([mon_features_df, unmon_features_df], ignore_index=True)

X = features_df[FEATURES_VER3]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
28995                    32.09               0.163669   
28996                    38.62               0.114350   
28997                    34.93               1.331199   
28998                    11.84               0.083521   
28999                     9.62               0.026874   

       avg_outgoing_burst_size  avg_incoming_burst_size  num_outgoing_packets  \
0                     1.551282                16.666667                 121.0   
1                     1.702128                 9.319149                  80.0   
2                     1.552632                16.315789                 

### 2. SVM

Ignore ConvergenceWarning

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

Split the dataset into training and testing sets

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Map all points to have mean=0 and std=1

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train.values)
X_test_scale = scaler.transform(X_test.values)
X_scale = scaler.fit_transform(X.values)

Train and test SVM using RBF kernel

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

clf_rbf = SVC(kernel='rbf', C=100, gamma=0.1, class_weight='balanced')
clf_rbf.fit(X_train_scale, y_train)
y_pred_rbf = clf_rbf.predict(X_test_scale)

print("============ Before Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_rbf)))
print("Classification Report:\n", classification_report(y_test, y_pred_rbf))

============ Before Hyperparameter tuning ============
SVM Accuracy: 0.8208275862068966
Classification Report:
               precision    recall  f1-score   support

          -1       0.96      0.83      0.89      2470
           0       0.74      0.76      0.75        42
           1       0.89      0.80      0.85        51
           2       0.57      0.96      0.71        47
           3       0.67      0.94      0.78        51
           4       0.64      0.79      0.71        43
           5       0.86      0.86      0.86        44
           6       0.83      0.92      0.88        53
           7       0.80      0.65      0.72        55
           8       0.79      0.72      0.76        47
           9       0.63      0.71      0.67        45
          10       0.63      0.75      0.69        44
          11       0.66      0.82      0.73        55
          12       0.74      0.94      0.83        52
          13       0.57      0.54      0.56        57
          14       0.69

### 3. Hyperparameter tuning by using Grid Search

In [6]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

search = HalvingGridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5, factor=3, n_jobs=-1)
search.fit(X_train_scale, y_train)

print(search.best_params_)
print(search.best_estimator_)

{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
SVC(C=100, class_weight='balanced', gamma=0.1)


In [7]:
y_pred_tuned = search.predict(X_test_scale)

print("============ After Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_tuned)))
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))

============ After Hyperparameter tuning ============
SVM Accuracy: 0.8208275862068966
Classification Report:
               precision    recall  f1-score   support

          -1       0.96      0.83      0.89      2470
           0       0.74      0.76      0.75        42
           1       0.89      0.80      0.85        51
           2       0.57      0.96      0.71        47
           3       0.67      0.94      0.78        51
           4       0.64      0.79      0.71        43
           5       0.86      0.86      0.86        44
           6       0.83      0.92      0.88        53
           7       0.80      0.65      0.72        55
           8       0.79      0.72      0.76        47
           9       0.63      0.71      0.67        45
          10       0.63      0.75      0.69        44
          11       0.66      0.82      0.73        55
          12       0.74      0.94      0.83        52
          13       0.57      0.54      0.56        57
          14       0.69 